**Función para dividir una tabla y obtener una base para SMS y Email**

In [ ]:
#librerias
import pandas as pd

**Función**

In [2]:
def procesar_archivo(archivo_fuente, email_output, sms_output):
    """
    Procesa un archivo CSV para limpiar nombres, correos y teléfonos, y genera dos archivos:
    - SMS.csv: Contiene CUENTA, NOMBRES y TELEFONO (sin correos).
    - EMAIL.csv: Contiene CUENTA, NOMBRES y EMAIL (solo correos válidos).

    Parámetros:
    archivo_fuente (str): Ruta del archivo CSV de entrada.
    """
    # Cargar el archivo
    df = pd.read_csv(archivo_fuente)
    print(f"Número total de registros en el archivo: {len(df)}")
    print()

    # Limpiar la columna 'NOMBRES'
    print("SECCIÓN DE LIMPIEZA DE NOMBRES")
    print(f"Número de registros antes de limpiar 'NOMBRES': {len(df)}")
    df['NOMBRES'] = (
        df['NOMBRES']
        .fillna("ESTIMADO")  # Llenar valores nulos con "ESTIMADO"
        .astype(str)  # Asegurarse de que todos los valores sean strings
        .str.strip()  # Eliminar espacios al inicio y final
        .str.split(" ").str[0]  # Tomar solo el primer nombre
        .str.replace(r"[^a-zA-Z]", "", regex=True)  # Eliminar caracteres no alfabéticos
        .replace("MA", "MARIA")  # Reemplazar "MA" por "MARIA"
    )
    # Si el nombre es muy corto, reemplazarlo por "ESTIMADO"
    df.loc[df['NOMBRES'].str.len() < 3, 'NOMBRES'] = "ESTIMADO"
    print(f"Número de nombres llenados con 'ESTIMADO': {(df['NOMBRES'] == 'ESTIMADO').sum()}")
    print(f"Número de registros después de limpiar 'NOMBRES': {len(df)}")
    print()

    # Limpiar la columna 'EMAIL'
    print("SECCIÓN DE LIMPIEZA DE CORREOS")
    print(f"Número de registros antes de limpiar 'EMAIL': {len(df)}")
    df['EMAIL'] = (
        df['EMAIL']
        .fillna("")  # Llenar valores nulos con cadena vacía
        .astype(str)  # Asegurarse de que todos los valores sean strings
        .str.strip()  # Eliminar espacios al inicio y final
        .str.lower()  # Convertir a minúsculas
        .str.replace(" ", "")  # Eliminar espacios intermedios
    )
    # Filtrar solo correos no vacíos
    df = df[df['EMAIL'] != ""]
    print(f"Número de registros después de limpiar 'EMAIL': {len(df)}")
    print()

    # Crear DataFrame SMS
    print("SECCIÓN DE CREACIÓN DE TABLA SMS")
    df_sms = df[['CUENTA', 'NOMBRES', 'TELEFONO']].copy()
    print(f"Número de registros antes de filtrar teléfonos: {len(df_sms)}")
    # Aplicar split en la columna 'TELEFONO' y tomar solo la primera parte
    df_sms['TELEFONO'] = df_sms['TELEFONO'].astype(str).str.split(" ").str[0]

    # Filtrar teléfonos válidos (10 dígitos y no seriados)
    df_sms = df_sms[
        (df_sms['TELEFONO'].str.len() == 10) &  # Teléfonos de 10 dígitos
        (~df_sms['TELEFONO'].str.fullmatch(r'(\d)\1{9}'))  # No teléfonos seriados
    ]
    print(f"Número de registros después de filtrar teléfonos: {len(df_sms)}")
    print()

    # Crear DataFrame EMAIL
    print("SECCIÓN DE CREACIÓN DE TABLA EMAIL")
    df_email = df[['CUENTA', 'NOMBRES', 'EMAIL']].copy()
    print(f"Número de registros antes de filtrar correos: {len(df_email)}")
    # Filtrar solo correos no vacíos
    df_email = df_email[df_email['EMAIL'] != ""]
    print(f"Número de registros después de filtrar correos: {len(df_email)}")
    print()

    # Exportar archivos
    df_sms.to_csv(sms_output, index=False)
    df_email.to_csv(email_output, index=False)

    print("Archivos SMS.csv y EMAIL.csv generados correctamente.")

**Uso**

In [3]:
# Uso de la función
procesar_archivo(
    r'C:\Users\lgonzalezc\Documents\Trabajo\DB\DOMICILIACION 200\BASE_EXTRA\Base extra cobranza.csv',
    r'C:\Users\lgonzalezc\Documents\Trabajo\DB\DOMICILIACION 200\BASE_EXTRA\bd_sms.csv',
    r'C:\Users\lgonzalezc\Documents\Trabajo\DB\DOMICILIACION 200\BASE_EXTRA\bd_email.csv'
)

SyntaxError: invalid syntax (2444976922.py, line 1)